# 卷积神经网络（CNN）
TensorFlow 2.0 实现，用卷积神经网络（CNN）完成 MNIST 手写体数字图片数据集的分类任务

In [1]:
import numpy as np
import tensorflow as tf

## 定义数据（MNIST）获取类

In [2]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

## 定义 CNN 模型

In [3]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(
            filters=32,             # 卷积层神经元（卷积核）数目
            kernel_size=[5, 5],     # 感受野大小
            padding='same',         # padding策略（vaild 或 same）
            activation=tf.nn.relu   # 激活函数
        )
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.conv2 = tf.keras.layers.Conv2D(
            filters=64,
            kernel_size=[5, 5],
            padding='same',
            activation=tf.nn.relu
        )
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size=[2, 2], strides=2)
        self.flatten = tf.keras.layers.Reshape(target_shape=(7 * 7 * 64,))
        self.dense1 = tf.keras.layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.conv1(inputs)                  # [batch_size, 28, 28, 32]
        x = self.pool1(x)                       # [batch_size, 14, 14, 32]
        x = self.conv2(x)                       # [batch_size, 14, 14, 64]
        x = self.pool2(x)                       # [batch_size, 7, 7, 64]
        x = self.flatten(x)                     # [batch_size, 7 * 7 * 64]
        x = self.dense1(x)                      # [batch_size, 1024]
        x = self.dense2(x)                      # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

## 模型训练

### 定义一些超参数

In [4]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001
display_step = 500

### 模型训练

In [5]:
model = CNN()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate) # Adam 优化器

In [6]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(1, num_batches):
    X, y = data_loader.get_batch(batch_size) # 训练数据集中随机抽取一批数据
    with tf.GradientTape() as tape:
        y_pred = model(X) 
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred) # 使用 tf.keras.losses 中的交叉熵函数作为损失函数
        loss = tf.reduce_mean(loss)
        if batch_index == 1 or batch_index % display_step == 0:
            print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables) # 计算损失函数关于模型变量的导数
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables)) # 将求出的导数值传入优化器，使用优化器的 apply_gradients 方法更新模型参数以最小化损失函数

batch 1: loss 2.309202
batch 500: loss 0.169802
batch 1000: loss 0.052987
batch 1500: loss 0.000710
batch 2000: loss 0.013420
batch 2500: loss 0.032587
batch 3000: loss 0.003149
batch 3500: loss 0.048259
batch 4000: loss 0.007030
batch 4500: loss 0.063868
batch 5000: loss 0.011883
batch 5500: loss 0.002714


## 模型评估

In [7]:
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model.predict(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.988700
